# Introduction

Moved from vectors that represent the appearance or absence of a word in a document like one-hot, which actually have a great representation of document because it keeps the order of the words in the document which keep the grammar, but it consumes a lot to process even small corpus, also the count of a word in the document have a great turned this memory usage in One-hot to just represent the count of the word in the document, but we know that some words appear a lot than others.

But now we need more information about the content that the words share in some document, like word frequency related to the other document, and to represent something like that we need continuous value to represent the word, then later we see how to represent these numbers in a way the represent not the importance of the word, but the meaning of the word it self.

- TF-IDF: Term Frequency **Times** Inverse Document Frequency.

We will see:

- Bag-of-Words: Word counts or Frequency.
- Bag-of-n-grams: Count of word pairs(Bigram) and 3-grams (triplets) and so on.
- TF-IDF: Word score that better represents their **importance** not **meaning**.

### *Lets go first in the first part **TF**.*

In [1]:
from nltk.tokenize import TreebankWordTokenizer
from collections import Counter
from nlpia.data.loaders import kite_text, kite_history
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

from collections import OrderedDict
import copy
import numpy as np
import math
nltk.download('brown')
from nltk.corpus import brown


/home/abdelrahman/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/abdelrahman/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/abdelrahman/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/abdelrahman/.local/lib/python3.6/site-packages/tensorflow/python/frame

In [2]:
sentence = """The faster Harry got to the store, the faster Harry, the faster, would get home."""
tokenizer = TreebankWordTokenizer()
tokens = tokenizer.tokenize(sentence.lower())
tokens

['the',
 'faster',
 'harry',
 'got',
 'to',
 'the',
 'store',
 ',',
 'the',
 'faster',
 'harry',
 ',',
 'the',
 'faster',
 ',',
 'would',
 'get',
 'home',
 '.']

In [3]:
bag_of_words = Counter(tokens)
bag_of_words

Counter({'the': 4,
         'faster': 3,
         'harry': 2,
         'got': 1,
         'to': 1,
         'store': 1,
         ',': 3,
         'would': 1,
         'get': 1,
         'home': 1,
         '.': 1})

In [4]:
bag_of_words.most_common(5) # method associated with counter class, return most repeated words.

[('the', 4), ('faster', 3), (',', 3), ('harry', 2), ('got', 1)]

# Note !

We can not assume that words that appear more times are more important to document than others, as we can see some tokens like [the, ,] and for an application like sentiment analysis words like [better, good] does not mean that the document is positive, what about the intent of the meaning.

Also using **python dictionary** or libraries like Collections, can shuffle the order of the word appearance, may be we can still capture the meaning after this shuffle but what about the long document.

The length of the document relative to the TF (Term Frequency) of that word in the document is important, because of we have multiple documents, it happens to have the **same** word little than in other document but its more important, this is because of the length of the document.

Suppose doc A have word **cat** appear 3 times in len doc of 30,
and doc B have word also **cat** but appear 100 times in len of 580000.

- Doc A is **email veterinarian**
- Doc B is **War  & Peace**

In [5]:
doc_a_cat = 3/30
doc_b_cat = 100/580000
print(doc_a_cat)
print(round(doc_b_cat,5))

0.1
0.00017


# Note !


So instead of **TF** (3 or 100 times) we can have actually the **importance of the word to document** using **Normalized Term Frequency** (.1 or .00017) frequence actually its probability of word not frequency.

In [6]:
times_harry_appears = bag_of_words['harry']
num_unique_words    = len(bag_of_words)
round(times_harry_appears/num_unique_words, 5)

0.18182

# Normalized TF

By using Normalized TF, we can see that describes something about the two document and their relationship to word cat and we can do for any word !, so instead of just raw count (TF), normlized these counts give you relative information.

So we can get each word and how it's relatively important to the document.

Let's move to a bigger piece of text.

In [7]:
# Lets look at some words of Bag-of-words we have
kite_text # Our Corups

"A kite is traditionally a tethered heavier-than-air craft with wing surfaces that react against the air to create lift and drag. A kite consists of wings, tethers, and anchors. Kites often have a bridle to guide the face of the kite at the correct angle so the wind can lift it. A kite's wing also may be so designed so a bridle is not needed; when kiting a sailplane for launch, the tether meets the wing at a single point. A kite may have fixed or moving anchors. Untraditionally in technical kiting, a kite consists of tether-set-coupled wing sets; even in technical kiting, though, a wing in the system is still often called the kite.\n\nThe lift that sustains the kite in flight is generated when air flows around the kite's surface, producing low pressure above and high pressure below the wings. The interaction with the wind also generates horizontal drag along the direction of the wind. The resultant force vector from the lift and drag force components is opposed by the tension of one or

In [8]:
tokens = tokenizer.tokenize(kite_text.lower()) # Tokenization
bag_of_words = Counter(tokens) # Get unique words and their counts
print(len(tokens))
print(len(bag_of_words))

363
180


In [9]:
print(bag_of_words['kite.'])
print(bag_of_words['kite'])
print(bag_of_words['water.'])

1
16
1


# Stope Words

A lot of stop words we have andy maybe it does not add any information as the text is about articles.

In [10]:
nltk.download("stopwords")
stop_words = nltk.corpus.stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/abdelrahman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
tokens = [token for token in tokens if token not in stop_words]
kite_counts = Counter(tokens)
print(len(tokens))
print(len(kite_counts))

222
147


In [12]:
kite_counts # We can see the difference after remove stopwords how its reduction the size of Bag-of-word we have

Counter({'kite': 16,
         'traditionally': 1,
         'tethered': 2,
         'heavier-than-air': 1,
         'craft': 2,
         'wing': 5,
         'surfaces': 1,
         'react': 1,
         'air': 2,
         'create': 1,
         'lift': 4,
         'drag.': 1,
         'consists': 2,
         'wings': 1,
         ',': 15,
         'tethers': 2,
         'anchors.': 2,
         'kites': 8,
         'often': 2,
         'bridle': 2,
         'guide': 1,
         'face': 1,
         'correct': 1,
         'angle': 1,
         'wind': 2,
         'it.': 1,
         "'s": 2,
         'also': 3,
         'may': 4,
         'designed': 2,
         'needed': 1,
         ';': 2,
         'kiting': 3,
         'sailplane': 1,
         'launch': 1,
         'tether': 1,
         'meets': 1,
         'single': 1,
         'point.': 1,
         'fixed': 1,
         'moving': 2,
         'untraditionally': 1,
         'technical': 2,
         'tether-set-coupled': 1,
         'sets': 1,

In [13]:
print(bag_of_words['kite.'])
print(bag_of_words['lift'])
print(bag_of_words['wing'])

1
4
5


# TreebankWordTokenizer

It return "kite." and "kite" as different tokens, it assume that your text segmented into sentences or documents, so it's only ignore punctuations at the very ending of strings when comes(\n).

sentences segmentation is tricky, we learn about later.

Spacy is working better for now, because it is due to sentence segmentation and tokenization.

## From Text to numbers what next ?

We have turned the textual data into numbers, but now instead of just count the word in the vector, we need values in the vector to be relative to something consistent across all vectors. For math operations is like to have the same scale(diemntion), share the same orgin.

So we do:
- Normalization
- Share same length or diemntion of all vectors
- want the value of each element of the vector to represent the same word in each document's Vector, so its fine to have 0 in some position of absence words.


In [14]:
# To Normalized TF

normalized_tf = []
for key, val in kite_counts.most_common():
    normalized_tf.append(val/len(kite_counts))
normalized_tf

[0.10884353741496598,
 0.10204081632653061,
 0.05442176870748299,
 0.034013605442176874,
 0.027210884353741496,
 0.027210884353741496,
 0.02040816326530612,
 0.02040816326530612,
 0.02040816326530612,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.013605442176870748,
 0.006802721088435374,
 0.006802721088435374,
 0.006802721088435374,
 0.006802721088435374,
 0.006802721088435374,
 0.006802721088435374,
 0.006802721088435374,
 0.006802721088435374,
 0.006802721088435374,
 0.006802721088435374,
 0.006802721088435374,
 0.00680272108843

# Note !

With just one vector for one document math does not have anything to do !, so lets us get other documents.

In [15]:
from nlpia.data.loaders import harry_docs as docs

In [16]:
docs

['The faster Harry got to the store, the faster and faster Harry would get home.',
 'Harry is hairy and faster than Jill.',
 'Jill is not as hairy as Harry.']

In [17]:
doc_tokens = []
all_doc_tokens = []
for doc in docs:
    doc_tokens    += [sorted(tokenizer.tokenize(doc.lower()))]
    all_doc_tokens +=  sorted(tokenizer.tokenize(doc.lower()))
print(len(doc_tokens))
print(len(doc_tokens[0]))
print(len(all_doc_tokens))

3
17
33


In [18]:
# unique words
lexicon = sorted(set(all_doc_tokens))
print(len(lexicon))
lexicon

18


[',',
 '.',
 'and',
 'as',
 'faster',
 'get',
 'got',
 'hairy',
 'harry',
 'home',
 'is',
 'jill',
 'not',
 'store',
 'than',
 'the',
 'to',
 'would']

## Again

Now each of the documents will take the same shared length **18** and give 0 for word that absence in this document and Normalized Tf for appearance one.

In [19]:
zero_vector = OrderedDict((token, 0) for token in lexicon)
zero_vector

OrderedDict([(',', 0),
             ('.', 0),
             ('and', 0),
             ('as', 0),
             ('faster', 0),
             ('get', 0),
             ('got', 0),
             ('hairy', 0),
             ('harry', 0),
             ('home', 0),
             ('is', 0),
             ('jill', 0),
             ('not', 0),
             ('store', 0),
             ('than', 0),
             ('the', 0),
             ('to', 0),
             ('would', 0)])

## Map for each document

In [20]:
doc_vectors = []
for doc in docs:
    vec = copy.copy(zero_vector)
    tokens = tokenizer.tokenize(doc.lower())
    token_count = Counter(tokens)
    for key, val in token_count.items():
        vec[key] = val / len(lexicon)
    doc_vectors.append(vec)
doc_vectors

[OrderedDict([(',', 0.05555555555555555),
              ('.', 0.05555555555555555),
              ('and', 0.05555555555555555),
              ('as', 0),
              ('faster', 0.16666666666666666),
              ('get', 0.05555555555555555),
              ('got', 0.05555555555555555),
              ('hairy', 0),
              ('harry', 0.1111111111111111),
              ('home', 0.05555555555555555),
              ('is', 0),
              ('jill', 0),
              ('not', 0),
              ('store', 0.05555555555555555),
              ('than', 0),
              ('the', 0.16666666666666666),
              ('to', 0.05555555555555555),
              ('would', 0.05555555555555555)]),
 OrderedDict([(',', 0),
              ('.', 0.05555555555555555),
              ('and', 0.05555555555555555),
              ('as', 0),
              ('faster', 0.05555555555555555),
              ('get', 0),
              ('got', 0),
              ('hairy', 0.05555555555555555),
              ('harry', 0.05

## Note !

Now we have three vectors, one for each document and we can apply math operation, but lets take a time understand vectors and vectors space in math.

# Vector Space

Vector is primary building block in the linear algebra, these vectors are list of numbers or cordinates in vector space, they decribe a location or position in that space or they can be used to identify a particular direction and magnitude (length) or distance in that space. A space is the collection of all posible vectors that could apear in that space. So vector with 2 space values would lie in 2-d vector space and so on.

In NLP the space is all the unique words in our courps and more spaces means more computions power as when you search in list of 5 numbers not as in list of million number.

To represent the space of our example which 18-d we can not do it so there is something will know about later called dimension reduction to reduce your 18-d to 2-d or 3-d that you can display.

actually linear algebra work for 2-d space as same as any number of space but computation power be a large and distance between vectors be large.

<img src='1.jpg'>

As we can see we can do different math operations and equations to get the distance between two vectors and the angle between them in order to know the similarity.

<img src='2.png'>

It output range from -1 to 1 which represent how two vector are share same direction or not which in other word is these vector similar or not as we can see below.
In NLP 1 means that the two vector share the same words and between that is the angle from -1 to 1 as we get.

**when its 0 then it means no words are common between two vector, and this means they talking about completely different things and actually not means they have different meaning or topics, just they use differnt words.**

It can not be -1 as we do not have TF of negative values, but in math it means they point in opposite dierctions, so in this case it will be in same space in case of 2-d same **quadrant**.

Later we can see negative values when we develop a concept of words and topics that are **opposite** to each other, then we will see cosine of negative values. !

In [21]:
def cosine_sim(vec1, vec2):
    result = ( np.matmul(vec1, vec2)) / (np.sum((np.linalg.norm(vec1))) * np.sum((np.linalg.norm(vec2))))
    
    return result

In [22]:
vec1 = np.array([.1, .17])
vec2 = np.array([.056, .056])
cosine_sim(vec1, vec2)

0.9679968981658008

In [23]:
cosine_sim(vec1, vec1)

1.0

## Zipfs Law

From the pattern when display the rank in descending order showed that. The log frequency of any word is inversely proportional to its rank in the frequency table, means that first item will appear twice as often the second, and three times as often the third and so on in the documents.

In [24]:
# Lets see brown courps from nltk
brown.words()[:10]

['The',
 'Fulton',
 'County',
 'Grand',
 'Jury',
 'said',
 'Friday',
 'an',
 'investigation',
 'of']

In [25]:
len(brown.words())

1161192

In [26]:
# lets remove the punctuations
import string

In [27]:
puncs = string.punctuation
puncs

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [28]:
word_list = [x.lower() for x in brown.words() if x not in puncs]
len(word_list)

1034378

In [29]:
token_counts = Counter(word_list)

In [30]:
token_counts.most_common(10)

[('the', 69971),
 ('of', 36412),
 ('and', 28853),
 ('to', 26158),
 ('a', 23195),
 ('in', 21337),
 ('that', 10594),
 ('is', 10109),
 ('was', 9815),
 ('he', 9548)]

# Topic Modeling

Return to TF and Normalized Term frequency that was normalized by the length of the Vocabulary size, is not tell us much more about the important of the word to a document that appear in **relative** to other documents rather than if we add the pattern of Zipfs law.

If you could suss out that information , you could describe document within the corpus.  Some words as we can see like "the" are likely to appear accross all documents while "construction" or "aerodynamics" less likely to see in the documents, and for those ones where frequently occured we need to know more about the nature of document it contain, so for this we need IDF (inverse document frequently) which dervitive from Zipfs law in topic (document) analysis. 

In [31]:
kite_intro = kite_text.lower()
intro_tokens = tokenizer.tokenize(kite_intro)
kite_history = kite_history.lower()
history_tokens = tokenizer.tokenize(kite_history)
intro_total = len(intro_tokens)
print(intro_total)
history_total = len(history_tokens)
print(history_total)

363
297


In [32]:
intro_tf = Counter(intro_tokens)
history_tf = Counter(history_tokens)

In [33]:
print(intro_tf['kite'] / intro_total)
print(history_tf['kite'] / history_total)

0.0440771349862259
0.020202020202020204


In [34]:
print(intro_tf['and'] / intro_total)
print(history_tf['and'] / history_total)

0.027548209366391185
0.030303030303030304


# Note !!


We can see that how the word kite was appeared twice in kite_intro rather than in kite history means that if we have much more documents its appeared would be less likely, and on the other side the word "and" was appeared much more in second document rather than the first one !

As we can see that we know can get more information about the important of word to the document as same to the other documents, because if it appear more in one documents but less likely in other documents means that its so important to this document on the other side if it much occurred across all documents like word "and", it will be less important to document.



**IDF** the ratio of total number of documents to the number of documents the word (term) appear in.

In [35]:
print(intro_tf['china'] / intro_total)
print(history_tf['china'] / history_total)

0.0
0.010101010101010102


#### Lets see and and china
#### Total document 2
#### and appears in the two document as we can see the normalized TF, so 2/2 = 1, two document/ appear in the two document
#### china appears just 1 as we can see the normalized TF, so 2/1 = 2, two document/ appear in the one document

** We can see who have  much weight than other **

So Now we can conclude that:
TF = Term frequency / document length.
IDF = All docs / Docs contain this term (count one for each document it appears in).

TF(t, d) = Count(t) / Count(d), 
- T is the word it self
- d is the number of words in the document

IDF(t, D) = log(number of documents / number of documents containing t)

Now the TF-IDF is by myltiply the two result

tf_idf(t,d, D) = tf * idf

# TF 

Lets look again step by step for tf.

In [36]:
intro_kite_tf = intro_tf['kite'] / intro_total # doc 1
history_kite_tf = history_tf['kite'] / history_total # doc 2

intro_and_tf = intro_tf['and'] / intro_total # doc 1
history_and_tf = history_tf['and'] / history_total # doc 2

intro_china_tf = intro_tf['china'] / intro_total # doc 1
history_china_tf = history_tf['china'] / history_total # doc 2

print(intro_kite_tf, history_kite_tf) 
print("="*50)
print(intro_and_tf, history_and_tf)
print("="*50)
print(intro_china_tf, history_china_tf)
print("="*50)

0.0440771349862259 0.020202020202020204
0.027548209366391185 0.030303030303030304
0.0 0.010101010101010102


# IDF 

Lets look again step by step for IDF.

We have a constant which is number of documents.

From above we can see that each print have two real numbers exept last one first number is 0 which means that the word not appeared in the first document, so we will devide by 2 for each word except last one as we just have 2 documents.

In [37]:
doc_number = 2

kite_idf = 2 / 2 # as it appeared in the two documents

and_idf = 2 / 2 # as it appeared in the two documents

china_idf = 2 / 1 # as it appeared in the just one document ????????

print(kite_idf)
print(and_idf)
print(china_idf)

1.0
1.0
2.0


In [38]:
print("The important of Kite in intro  document and accross other documents", (intro_kite_tf * kite_idf))
print("The important of Kite in history document  and accross other documents", (history_kite_tf * kite_idf))

print("The important of and in intro  document and accross other documents", (intro_and_tf * and_idf))
print("The important of and in history document  and accross other documents", (history_and_tf * and_idf))

print("The important of china in intro  document and accross other documents", (intro_china_tf * china_idf))
print("The important of china in history document  and accross other documents", (history_china_tf * china_idf))

The important of Kite in intro  document and accross other documents 0.0440771349862259
The important of Kite in history document  and accross other documents 0.020202020202020204
The important of and in intro  document and accross other documents 0.027548209366391185
The important of and in history document  and accross other documents 0.030303030303030304
The important of china in intro  document and accross other documents 0.0
The important of china in history document  and accross other documents 0.020202020202020204


# Note !

So the 0 in the fifth print is about that the word china actually does not appear in this document, so it has no weights.

if we have more documents we will see how other words are important to documents even they appear less likely than other words.

# Return to Zipfs

We have seen that in brown corpus as it semi large dataset we have the words appears large and large, and some words on the other side will just seen in small number of documents maybe 10 of million documents and other word just appeared 1, and in this case you will have idf = 1000,000 / 10 = 1000,00 and 1000,000 / 1 = 1000,000, even of that is just the distance is just in 10 documents but it exponentially higher frequency, so Sipfs suggest use the log which the inverse of exponential to return to the rigth state.

This ensure that TF-IDF scores are more uniformly distributed, so we can now define the IDF as the log of the original probability of that word occuring in one of your documents.



# Relevance Ranking

Before we use word count as our main feature and replace the word in each document with this count, but it does not give us more information about the important of this word to the document or topic it talks about. But now after some analysis and Zipf's law we have noticed some pattern to keeping going improve the information we get about the word it self and how its effect on the document not just this but also relevant to other documents.

In [39]:
documt_tfidf_vectors = []
for doc in docs:
    vec = copy.copy(zero_vector)
    tokens = tokenizer.tokenize(doc.lower())
    token_counts = Counter(tokens)
    
    for key, val in token_counts.items():
        docs_contain_key = 0
        for _doc in docs:
            if key in _doc:
                docs_contain_key +=1
        tf = val / len(lexicon)
        if docs_contain_key:
            idf = len(docs) / docs_contain_key
        else: idf = 0
        
        vec[key] = tf * idf
    documt_tfidf_vectors.append(vec)

In [40]:
documt_tfidf_vectors[0]

OrderedDict([(',', 0.16666666666666666),
             ('.', 0.05555555555555555),
             ('and', 0.08333333333333333),
             ('as', 0),
             ('faster', 0.25),
             ('get', 0.16666666666666666),
             ('got', 0.16666666666666666),
             ('hairy', 0),
             ('harry', 0.0),
             ('home', 0.16666666666666666),
             ('is', 0),
             ('jill', 0),
             ('not', 0),
             ('store', 0.16666666666666666),
             ('than', 0),
             ('the', 0.5),
             ('to', 0.16666666666666666),
             ('would', 0.16666666666666666)])

# Cosine similarity

Now you can get the angle between two vectors to know they similar or not at all, or even use it to check query on your documents and return the most similar documents based on the query.



In [41]:
query = "How long does not it take as to get to the store?"
query_vec = copy.copy(zero_vector)
tokens = tokenizer.tokenize(query.lower())
token_counts = Counter(tokens)

In [42]:
for key, val in token_counts.items():
    docs_contain_key = 0
    for _doc in docs:
        if key in _doc.lower():
            docs_contain_key +=1
    if docs_contain_key == 0:
        continue
    tf = val / len(tokens)
    idf = len(docs) / docs_contain_key
    query_vec[key] = tf * idf


In [43]:
q = np.array(list(query_vec.values()))
# q = q.reshape(-1, 1)
doc1 = np.array(list(documt_tfidf_vectors[0].values()))
doc2 = np.array(list(documt_tfidf_vectors[1].values()))
doc3 = np.array(list(documt_tfidf_vectors[2].values()))
# doc1 = doc1.reshape(-1,1)
cosine_sim(q, doc1)

0.5697334438987307

In [44]:
cosine_sim(q, doc2)

0.0

In [45]:
cosine_sim(q, doc3)

0.2993266539431952

# Tools
Instead of doing hard code for everything, a lot of APIs are available for us to use. But now we have the intuation beside of what we will use we know hot it works.

TfidfVectorizer return a sparce materix(most of 0) because we know each document is small number of words of the whole corpus.

Using **todense** convert the sparce metrix to numbey metrix

In [46]:
corpus = docs
vectorizer = TfidfVectorizer(min_df=1)
model = vectorizer.fit_transform(corpus)
print(model.todense().round(2))

[[0.16 0.   0.48 0.21 0.21 0.   0.25 0.21 0.   0.   0.   0.21 0.   0.64
  0.21 0.21]
 [0.37 0.   0.37 0.   0.   0.37 0.29 0.   0.37 0.37 0.   0.   0.49 0.
  0.   0.  ]
 [0.   0.75 0.   0.   0.   0.29 0.22 0.   0.29 0.29 0.38 0.   0.   0.
  0.   0.  ]]


# Sklearn

** As we can see how it simple using sklearn instead of all of the code above**

TfidfVectorizer have its own tokenizer so it maybe different from what we hand code, but now you have a matrix that represent each document, TF-IDF for each term in your lexicon(unique words).